In [1]:
import pandas as pd
import os
import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

## Day Ahead preliminary file name cleaning

In [17]:
os.chdir(r"C:\Users\jerep\Downloads\raw_data\day_ahead")

In [3]:
for file in os.listdir():
    if ".." in file:
        new_name = file.replace("..", ".")
        os.rename(file, new_name)

In [19]:
start_date = datetime.date(2010,1,1)
oneday = datetime.timedelta(days=1)

not_downloaded = []

for i in range(365):
    current_date = start_date + i * oneday
    current_filename = current_date.strftime("%Y%m%d") + "MGPOffertePubbliche.xml"
    if current_filename not in os.listdir(): not_downloaded.append(current_filename)

In [20]:
not_downloaded

[]

## Ancillary preliminary file name cleaning

In [2]:
os.chdir(r"C:\Users\jerep\Downloads\raw_data\ancillary")

In [4]:
for file in os.listdir():
    if ".." in file:
        print(file)

In [5]:
start_date = datetime.date(2010,1,1)
oneday = datetime.timedelta(days=1)

not_downloaded = []

for i in range(365):
    current_date = start_date + i * oneday
    current_filename = current_date.strftime("%Y%m%d") + "MSDOffertePubbliche.xml"
    if current_filename not in os.listdir(): not_downloaded.append(current_filename)

In [6]:
not_downloaded

[]

In [10]:
pd.read_xml(open("20100101MSDOffertePubbliche.xml")).iloc[1:, 2:]

,PURPOSE_CD,TYPE_CD,STATUS_CD,MARKET_CD,UNIT_REFERENCE_NO,INTERVAL_NO,BID_OFFER_DATE_DT,TRANSACTION_REFERENCE_NO,QUANTITY_NO,AWARDED_QUANTITY_NO,...,MERIT_ORDER_NO,PARTIAL_QTY_ACCEPTED_IN,ADJ_QUANTITY_NO,ADJ_ENERGY_PRICE_NO,ZONE_CD,AWARDED_PRICE_NO,OPERATORE,SUBMITTED_DT,BILATERAL_IN,SCOPE
1,BID,REG,REJ,MSD,UP_AGRI_1,1.0,20100101.0,9.365057e+13,0.0,0.0,...,1.0,N,0.0,0.0,SUD,0.0,ENEL PRODUZIONE S.P.A.,2.009123e+16,False,AS
2,BID,REG,REJ,MSD,UP_AGRI_1,1.0,20100101.0,9.365057e+13,0.0,0.0,...,1.0,N,0.0,0.0,SUD,0.0,ENEL PRODUZIONE S.P.A.,2.009123e+16,False,GR1
3,OFF,REG,REJ,MSD,UP_AGRI_1,1.0,20100101.0,9.365057e+13,0.0,0.0,...,1.0,N,0.0,0.0,SUD,114.0,ENEL PRODUZIONE S.P.A.,2.009123e+16,False,AS
4,OFF,REG,REJ,MSD,UP_AGRI_1,1.0,20100101.0,9.365057e+13,0.0,0.0,...,1.0,N,0.0,0.0,SUD,115.0,ENEL PRODUZIONE S.P.A.,2.009123e+16,False,GR1
5,BID,REG,REJ,MSD,UP_AGRI_1,2.0,20100101.0,9.365057e+13,0.0,0.0,...,1.0,N,0.0,0.0,SUD,0.0,ENEL PRODUZIONE S.P.A.,2.009123e+16,False,GR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,OFF,REG,REP,MSD,UP_VOGHERA_1,24.0,20100101.0,9.365057e+13,206.0,0.0,...,0.0,N,206.0,0.0,None,0.0,ACEAELECTRABEL TRADING S.P.A.,2.009123e+16,False,AS
37642,OFF,REG,REP,MSD,UP_VOGHERA_1,24.0,20100101.0,9.365057e+13,24.0,0.0,...,0.0,N,24.0,0.0,None,0.0,ACEAELECTRABEL TRADING S.P.A.,2.009123e+16,False,RS
37643,OFF,REG,REP,MSD,UP_VOGHERA_1,24.0,20100101.0,9.365057e+13,168.0,0.0,...,0.0,N,168.0,0.0,None,0.0,ACEAELECTRABEL TRADING S.P.A.,2.009123e+16,False,GR1
37644,OFF,REG,REP,MSD,UP_VOGHERA_1,24.0,20100101.0,9.365057e+13,206.0,0.0,...,0.0,N,206.0,0.0,None,0.0,ACEAELECTRABEL TRADING S.P.A.,2.009123e+16,False,AS


## Read all files by multi-threading
Multi-threading can be useful to greatly reduced the time it takes to read multiple xml files.

In [5]:
# iterate over dates
start_date = datetime.date(2010,1,1)
oneday = datetime.timedelta(days=1)

# worker function
def process_date(i):
    # processed date
    current_date = start_date + i * oneday
    current_filename = current_date.strftime("%Y%m%d") + "MGPOffertePubbliche.xml"

    # read xml
    temp_df = pd.read_xml(open(current_filename))

    # remove unnecessary readings
    temp_df = temp_df.iloc[1:, 2:]

    return temp_df

In [10]:
dfs = []

# use a threadpool to run concurrent worker threads
def main():
    with ProcessPoolExecutor() as executor:
        
        # submit all the tasks to the executor
        future_to_date = {executor.submit(process_date, i): i for i in range(365)}

        for future in tqdm(as_completed(future_to_date)):
            result = future.result()
            if result is not None:
                dfs.append(result)


    df = pd.concat(dfs, axis=0)

if __name__ == '__main__':
    main()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
df

In [70]:
df.to_csv("day_ahead.csv", index=False)

In [ ]:
pd.DataFrame()